In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns

### Distribution of Pupil Position and Diameter Between Trials

#### Load Data

In [4]:
dat = xr.load_dataset('data/steinmetz_2017-06-16_Hench.nc')
dat

<xarray.Dataset>
Dimensions:             (trial: 482, time: 250, cell: 788,
                         waveform_component: 3, sample: 82, probe: 384,
                         brain_area_lfp: 13, spike_id: 3666676)
Coordinates:
  * trial               (trial) int32 1 2 3 4 5 6 7 ... 477 478 479 480 481 482
  * time                (time) float64 0.01 0.02 0.03 0.04 ... 2.48 2.49 2.5
  * cell                (cell) int32 1 2 3 4 5 6 7 ... 783 784 785 786 787 788
  * waveform_component  (waveform_component) int32 1 2 3
  * probe               (probe) int32 1 2 3 4 5 6 7 ... 379 380 381 382 383 384
  * brain_area_lfp      (brain_area_lfp) object 'LSr' 'ORBm' ... 'VISam' 'VPL'
  * spike_id            (spike_id) int32 1 2 3 4 ... 3666674 3666675 3666676
Dimensions without coordinates: sample
Data variables: (12/31)
    contrast_left       (trial) int8 100 100 50 100 100 100 ... 0 0 0 100 100 0
    contrast_right      (trial) int8 25 0 0 50 0 25 25 ... 50 50 25 100 0 25 100
    gocue               (trial) float64 0.9236 0.8744 0.9076 ... nan nan nan
    stim_onset          (trial) float64 0.5 0.5 0.5 0.5 0.5 ... 0.5 0.5 0.5 0.5
    feedback_type       (trial) float64 1.0 1.0 1.0 1.0 1.0 ... nan nan nan nan
    feedback_time       (trial) float64 1.338 1.02 1.07 0.8708 ... nan nan nan
    ...                  ...
    waveform_w          (cell, sample, waveform_component) float32 0.0 ... -0...
    waveform_u          (cell, waveform_component, probe) float32 0.0 ... -0....
    lfp                 (brain_area_lfp, trial, time) float64 -1.251 ... 19.49
    spike_time          (spike_id) float32 0.886 1.87 0.2868 ... 0.8948 0.08402
    spike_cell          (spike_id) uint32 1 1 1 1 1 1 ... 788 788 788 788 788
    spike_trial         (spike_id) uint32 1 1 2 4 5 5 ... 396 431 445 453 456
Attributes:
    session_date:  2017-06-16
    mouse:         Hench
    stim_onset:    0.5
    bin_size:      0.01

#### Select Variables

We want the pupil data and which stimuli were present during that trial

In [23]:
df = (
    dat
    [['active_trials', 'contrast_left', 'contrast_right', 'pupil_x', 'pupil_y', 'pupil_area']]
    .to_dataframe()
    .reset_index()
)
df = df[df['active_trials'] == True].copy()
df.sample(10)

,trial,time,active_trials,contrast_left,contrast_right,pupil_x,pupil_y,pupil_area
42360,170,1.11,True,0,0,-0.397205,-0.377221,0.005139
69266,278,0.17,True,100,0,-0.323541,-0.116788,0.074480
4647,19,1.48,True,0,100,-0.621175,0.031184,0.017430
87394,350,1.45,True,0,50,-0.424741,0.582965,0.101940
87647,351,1.48,True,0,100,-0.710276,0.435928,0.128477
11534,47,0.35,True,0,100,-0.377566,-0.282868,0.021561
58719,235,2.20,True,0,0,-0.716673,0.520856,0.021157
62293,250,0.44,True,25,100,0.151187,0.028048,0.081474
61963,248,2.14,True,0,0,-0.575074,-0.054473,0.018966
20558,83,0.59,True,50,0,0.019213,0.315315,0.020488


### Categorize Trials
We'll keep only the trials where there was only 100% contrast on the left or 100% contrast on the right.

In [ ]:
query_left = (df['contrast_left'] == 100) & (df['contrast_right'] == 0)
query_right = (df['contrast_left'] == 0) & (df['contrast_right'] == 100)
dd = df[query_left | query_right ].copy()
dd['TrialType'] = 'Right'
dd.loc[dd['contrast_left'] == 100, "TrialType"] = "Left"
dd[['contrast_left', 'contrast_right']].value_counts()